In [1]:
from langchain.chat_models.gigachat import GigaChat
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import load_prompt
from langchain.schema import HumanMessage
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain_community.embeddings.gigachat import GigaChatEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts.chat import (
    AIMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [28]:
AUTH_TOKEN = "NTlkY2MyZmItM2Q4ZC00ZWMzLWE2NjAtNTI3MzZhOTk2ZjQzOjVhZGJiZDQxLTc0YjAtNDQxNi04YjAzLTUxZDVmYTY4NTkwNw=="
PDF_PATH = "/Users/ampiro/programs/HACKATONS/DeepHackCOGers/data/raw/PanovRL.pdf"

In [3]:
llm = GigaChat(credentials=AUTH_TOKEN, verify_ssl_certs=False, scope='GIGACHAT_API_CORP')
embberder = GigaChatEmbeddings(
    credentials=AUTH_TOKEN, verify_ssl_certs=False, scope='GIGACHAT_API_CORP'
)

In [29]:
loader = PyPDFLoader(PDF_PATH)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

Total documents: 229


In [30]:
db = Chroma.from_documents(
    documents,
    embberder,
    client_settings=Settings(anonymized_telemetry=False),
)

In [31]:
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

In [7]:
template = "Ты полезный ассистент, который умеет переводить {input_language} на {output_language}. Переведи следующее предложение."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

# get a chat completion from the formatted messages
llm.invoke(
    chat_prompt.format_prompt(
        input_language="русский",
        output_language="английский",
        text="Как представляется план",
    ).to_messages()
)

AIMessage(content='How is the plan presented?', response_metadata={'token_usage': Usage(prompt_tokens=37, completion_tokens=7, total_tokens=44), 'model_name': 'GigaChat:3.1.24.3', 'finish_reason': 'stop'}, id='run-38aca72e-c9b0-4f7d-9662-525b9a8d259c-0')

In [8]:
result = qa_chain({"query": "Show example across evaluated tasks"})["result"]
print(result)

/Users/ampiro/miniconda3/envs/cogers/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Here are the randomly chosen examples for the evaluated tasks:

Example 1: Task: Use computer
Step 1: Walk to home ofﬁce
Step 2: Walk to chair
Step 3: Find chair

Example 2: Task: Relax on sofa
Step 1: Walk to home ofﬁce
Step 2: Walk to couch
Step 3: Find couch
Step 4: Sit on couch
Step 5: Find pillow
Step 6: Lie on couch

Example 3: Task: Read book
Step 1: Walk to home ofﬁce
Step 2: Walk to couch
Step 3: Find couch
Step 4: Sit on couch
Step 5: Find pillow
Step 6: Lie on couch


In [9]:
qa_chain({"query": ""})

{'query': '', 'result': 'Не знаю'}

In [34]:
qa_chain.invoke({"query": "Какова основная идея алгоритма SARSA?"})

{'query': 'Какова основная идея алгоритма SARSA?',
 'result': 'Основная идея алгоритма SARSA заключается в использовании таблицы Q-значений для обучения агентов в задачах с подкреплением. Алгоритм использует переменные состояния, действия, текущее состояние (𝑠), предыдущее действие (𝑎), следующее состояние (𝑟) и следующее действие (𝑎′). Он обновляет таблицу Q-значений, используя функцию исследования среды 𝑓(𝑣, 𝑛) и переменную 𝛼.'}

In [11]:
qa_chain({"query": "В чем разница между TD подходом и SARSA?"})

{'query': 'В чем разница между TD подходом и SARSA?',
 'result': 'Извините, но я не могу ответить на ваш вопрос на русском языке. Можете ли вы повторить его на английском?'}

In [12]:
qa_chain({"query": "Что сделать девушке на ужин?"})

{'query': 'Что сделать девушке на ужин?',
 'result': 'Девушке на ужин можно приготовить различные блюда, например, запеченную курицу с картофелем, овощной салат или пиццу. Также можно заказать еду на дом или пойти в ресторан.'}

In [13]:
table_request = """
Ты ассистент помогающий писать обзоры к научным статьям. Тебя попросили заполнить таблицу со столбцами, обозначающие разные важные части статьи. Сформулируй запрос на получение всей информации из статьи по названию определенного столбца.
Название интересующего столбца: среда/датасет
Не используй слово столбец в тексте запроса.
Формат ответ должен выглядеть вот так:


Запрос: "текст запроса"


"""

In [14]:
templated = """QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER:""".format(
    "Как можно ",
    ""
)

KeyError: 'question'

In [15]:
qa_chain({"query": templated})

NameError: name 'templated' is not defined

In [34]:
template = """Ты делаешь обзор научных статей. Перед тобой название одной из частей обзора.
Придумай и напиши 3 УНИКАЛЬНЫХ КОРОТКИХ ВОПРОСА относящиеся к конкретной статье по названию части обзора.
Формат ответ должен выглядеть вот так:
Вопрос 1
Вопрос 2

Название части обзора: "{}"
3 вопроса к документу на русском языке:
"""

res = llm.invoke(template.format("параметры обучения"))
print(res.content)

1. Какие параметры обучения используются в данной статье?
2. Как эти параметры влияют на эффективность обучения?
3. Какие методы оптимизации параметров обучения рассматриваются в статье?


In [17]:
command = "расскажи о чем статья абоба, обязательно скажи про dataset, llm, plan structure"

llm.invoke("""Ты многофункциональный ассистент. Ты можешь выполнять следующие действия [добавить статью в таблицу, написать короткое описание, написать короткое структурированное описание, сравнить две статьи].
Пользователь попросил тебя выполнить команду: "".
К какому из этих действий запрос пользователя ближе всего:
1. Добавить статью в таблицу
2. Написать СТРУКТУРИРОВАННЫЙ обзор на статью
3. Написать обзор на статью
4. Сравнить две статьи

Ответь только номером действия из списка. Номер ближайшего к завпросу действия:
""")

AIMessage(content='3', response_metadata={'token_usage': Usage(prompt_tokens=148, completion_tokens=3, total_tokens=151), 'model_name': 'GigaChat:3.1.24.3', 'finish_reason': 'stop'}, id='run-17eea641-0780-433a-9013-53596fb23196-0')

In [23]:
import re
s = "1. Написать обзор на статью"
print()


1


{}

In [38]:
req = [
    "Напиши мне обзор на статью Attention is all you need, выделяя эти факторы: датасет, языковая модель, пример плана действий, название статьи",
    "расскажи какие датасеты, какая llm, какие параметры обучения были использованны в работе Абоба",
    "хочу знать какой датасет был использованн в статье 'Сибирские котята'"
]

In [42]:
print(llm.invoke("""
Ты ассистент, помогающий пользователю выделять наименованные сущности из текста. Пользователь попросил написать обзор на статью и назвал несколько пунктов, его интересующих. Определи и верни пользователю в качестве ответа.
Пример ответа:
Сущность 1, сущность 2, сущность3.
           
Запрос от пользователя: "{}"
Список выделенных сущностей:
""".format(req[0])).content)

Сущность 1: датасет
Сущность 2: языковая модель
Сущность 3: пример плана действий
Сущность 4: название статьи


In [35]:
llm.invoke("У нас с тобой получилось выполнить команду! Скажи что-нибудь радостное)")

AIMessage(content='Ура! Мы это сделали! Хочу сказать, что я очень рада, что мы смогли выполнить эту команду вместе. Это было весело и увлекательно. Спасибо за компанию!', response_metadata={'token_usage': Usage(prompt_tokens=27, completion_tokens=37, total_tokens=64), 'model_name': 'GigaChat:3.1.24.3', 'finish_reason': 'stop'}, id='run-e52d576b-42b6-4cd6-bdfa-6ec8b8e670e6-0')

In [37]:
llm.invoke("У нас с тобой не получилось выполнить команду... Объяви этот факт и скажи что-нибудь утешающее)")

AIMessage(content='К сожалению, у нас не получилось выполнить команду. Но не переживайте, мы будем стараться изо всех сил и учиться вместе. Всегда есть возможность попробовать снова и достичь успеха.', response_metadata={'token_usage': Usage(prompt_tokens=34, completion_tokens=41, total_tokens=75), 'model_name': 'GigaChat:3.1.24.3', 'finish_reason': 'stop'}, id='run-860afc9b-c486-4c1c-92f3-72e687ed6c67-0')

In [16]:
book_map_prompt = load_prompt("lc://prompts/summarize/map_reduce/summarize_book_map.yaml")
book_combine_prompt = load_prompt("lc://prompts/summarize/map_reduce/summarize_book_combine.yaml")

chain = load_summarize_chain(llm, chain_type="map_reduce", 
                             map_prompt=book_map_prompt,
                             combine_prompt=book_combine_prompt,
                             verbose=False)


In [20]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=7000,
    chunk_overlap=500,
)

loader = PyPDFLoader(PDF_PATH)
documents = loader.load()
documents = text_splitter.split_documents(documents)
print(len(documents))

33


In [21]:
res = chain.invoke({"input_documents": documents})
print(res["output_text"].replace(". ", ".\n"))

Книга исследует возможность использования знаний, полученных от больших языковых моделей, для выполнения задач в интерактивных средах.
Авторы предлагают метод, который позволяет преобразовывать высокоуровневые задачи на естественном языке в планы действий, состоящие из нескольких шагов.
Они также показывают, что при правильном подходе LLM могут эффективно декомпозировать задачи без дополнительного обучения.
Однако планы, сгенерированные LLM, часто не соответствуют допустимым действиям.
Авторы предлагают процедуру, которая использует существующие демонстрации и переводит планы в допустимые действия.
Результаты оценки в виртуальной среде VirtualHome показывают, что этот метод значительно улучшает выполнение задач по сравнению с базовым LLM.


In [24]:
print(res["output_text"].replace(". ", ".\n"))

Книга исследует возможность использования знаний, полученных от больших языковых моделей, для выполнения задач в интерактивных средах.
Авторы предлагают метод, который позволяет преобразовывать высокоуровневые задачи на естественном языке в планы действий, состоящие из нескольких шагов.
Они также показывают, что при правильном подходе LLM могут эффективно декомпозировать задачи без дополнительного обучения.
Однако планы, сгенерированные LLM, часто не соответствуют допустимым действиям.
Авторы предлагают процедуру, которая использует существующие демонстрации и переводит планы в допустимые действия.
Результаты оценки в виртуальной среде VirtualHome показывают, что этот метод значительно улучшает выполнение задач по сравнению с базовым LLM.


In [26]:
res = chain.invoke({"input_documents": documents, "map_size": "сделай обзор на статью Абоба в 5 предложений"})


In [27]:
print(res["output_text"].replace(". ", ".\n"))

Книга рассматривает использование языковых моделей для выполнения задач в интерактивных средах, создания и выполнения действий в виртуальной среде, оценки правильности программ с помощью метрик, анализа длины программ и сравнения различных моделей машинного обучения.
В каждой части подробно описываются шаги, необходимые для успешного выполнения задачи.


In [35]:
from langchain.chains.combine_documents import create_stuff_documents_chain


question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)


from langchain.memory import ChatMessageHistory

demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("how can langsmith help with testing?")

document_chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
        "context": docs,
    }
)

NameError: name 'document_chain' is not defined